In [1]:
#%pip install super-gradients
#!pip install torch
#%pip install supervision

from roboflow import Roboflow
rf = Roboflow(api_key="NNsjGsIb23GWj3zGDggC")
project = rf.workspace("iot-inatel").project("braintumor-spfys")
dataset = project.version(4).download("yolov8")

loading Roboflow workspace...
loading Roboflow project...
Dependency ultralytics<=8.0.20 is required but found version=8.0.168, to fix: `pip install ultralytics<=8.0.20`


Extracting Dataset Version Zip to BrainTumor-4 in yolov8:: 100%|██████████| 1222/1222 [00:00<00:00, 1388.31it/s]


In [3]:
import torch
from super_gradients.training import Trainer

dataset.location = 'c:/Users/mateu/ICC/CS&I-LAB/YOLO-NAS-Tumor/BrainTumor-4'
DEVICE = 'cuda' if torch.cuda.is_available() else "cpu"
MODEL_ARCH = 'yolo_nas_l'
ATCH_SIZE = 8
MAX_EPOCHS = 10
CHECKPOINT_DIR = './checkpoints'
EXPERIMENT_NAME = project.name.lower().replace(" ", "_")
LOCATION = dataset.location
CLASSES = sorted(project.classes.keys())

dataset_params = {
    'data_dir': LOCATION,
    'train_images_dir':'train/images',
    'train_labels_dir':'train/labels',
    'val_images_dir':'valid/images',
    'val_labels_dir':'valid/labels',
    'test_images_dir':'test/images',
    'test_labels_dir':'test/labels',
    'classes': CLASSES
}



In [7]:
from super_gradients.training.dataloaders.dataloaders import (coco_detection_yolo_format_train, coco_detection_yolo_format_val)

train_data = coco_detection_yolo_format_train(
    dataset_params={
        'data_dir': dataset_params['data_dir'],
        'images_dir': dataset_params['train_images_dir'],
        'labels_dir': dataset_params['train_labels_dir'],
        'classes': dataset_params['classes']
    },
    dataloader_params={
        'batch_size': 32,
        'num_workers': 2
    }
)

val_data = coco_detection_yolo_format_val(
    dataset_params={
        'data_dir': dataset_params['data_dir'],
        'images_dir': dataset_params['val_images_dir'],
        'labels_dir': dataset_params['val_labels_dir'],
        'classes': dataset_params['classes']
    },
    dataloader_params={
        'batch_size': 32,
        'num_workers': 2
    }
)

[2023-09-01 22:03:05] INFO - detection_dataset.py - Dataset Initialization in progress. `cache_annotations=True` causes the process to take longer due to full dataset indexing.
Indexing dataset annotations: 100%|██████████| 1184/1184 [00:11<00:00, 104.38it/s]
[2023-09-01 22:03:16] INFO - detection_dataset.py - Dataset Initialization in progress. `cache_annotations=True` causes the process to take longer due to full dataset indexing.
Indexing dataset annotations:  94%|█████████▍| 319/338 [00:03<00:00, 97.10it/s] 

Indexing dataset annotations: 100%|██████████| 338/338 [00:03<00:00, 101.53it/s]


In [8]:
from super_gradients.training import models

model = models.get(
    MODEL_ARCH, 
    num_classes=len(dataset_params['classes']), 
    pretrained_weights="coco"
)

[2023-09-01 22:03:25] INFO - checkpoint_utils.py - License Notification: YOLO-NAS pre-trained weights are subjected to the specific license terms and conditions detailed in 
https://github.com/Deci-AI/super-gradients/blob/master/LICENSE.YOLONAS.md
By downloading the pre-trained weight files you agree to comply with these terms.
[2023-09-01 22:03:25] INFO - checkpoint_utils.py - Successfully loaded pretrained weights for architecture yolo_nas_l


In [56]:
from super_gradients.training.losses import PPYoloELoss
from super_gradients.training.metrics import DetectionMetrics_050
from super_gradients.training.models.detection_models.pp_yolo_e import PPYoloEPostPredictionCallback

train_params = {
    'silent_mode': False,
    "average_best_models":True,
    "warmup_mode": "linear_epoch_step",
    "warmup_initial_lr": 1e-6,
    "lr_warmup_epochs": 3,
    "initial_lr": 5e-4,
    "lr_mode": "cosine",
    "cosine_final_lr_ratio": 0.1,
    "optimizer": "Adam",
    "optimizer_params": {"weight_decay": 0.0001},
    "zero_weight_decay_on_bias_and_bn": True,
    "ema": True,
    "ema_params": {"decay": 0.9, "decay_type": "threshold"},
    "max_epochs": MAX_EPOCHS,
    "mixed_precision": True,
    "loss": PPYoloELoss(
        use_static_assigner=False,
        num_classes=len(dataset_params['classes']),
        reg_max=16
    ),
    "valid_metrics_list": [
        DetectionMetrics_050(
            score_thres=0.1,
            top_k_predictions=300,
            num_cls=len(dataset_params['classes']),
            normalize_targets=True,
            post_prediction_callback=PPYoloEPostPredictionCallback(
                score_threshold=0.01,
                nms_top_k=1000,
                max_predictions=300,
                nms_threshold=0.7
            )
        )
    ],
    "metric_to_watch": 'mAP@0.50'
}

In [57]:
print(DEVICE)
trainer.train(
    model=model, 
    training_params=train_params, 
    train_loader=train_data, 
    valid_loader=val_data
)

cuda
The console stream is now moved to ./checkpoints\unreal_ships_detection/console_set01_21_48_40.txt


[2023-09-01 21:48:42] INFO - sg_trainer.py - Using EMA with params {'decay': 0.9, 'decay_type': 'threshold'}


AssertionError: mixed precision is not available for CPU

torch.cuda.amp.GradScaler is enabled, but CUDA is not available.  Disabling.


In [1]:
import torch

if torch.cuda.is_available():
    device = torch.device("cuda")
    print("GPU disponível.")
    print("Dispositivo atual:", torch.cuda.get_device_name(0))  # 0 é o índice da GPU, pode ser diferente
    print("Memória VRAM disponível:", torch.cuda.get_device_properties(0).total_memory / (1024 ** 2), "MB")
else:
    print("GPU não disponível.")


GPU não disponível.


In [7]:
%pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu117

Looking in indexes: https://download.pytorch.org/whl/cu117
Note: you may need to restart the kernel to use updated packages.
